In [62]:
%load_ext autoreload 
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dists
from src.api import *
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
x = torch.zeros(16, 1, 28, 28)
z_dim = 100
e = Chain([
    nn.Conv2d(1, 8, kernel_size=5, padding=2),
    nn.LeakyReLU(),
    nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1),
    nn.LeakyReLU(),
    nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
    nn.LeakyReLU(),
    lambda x: torch.flatten(x, start_dim=1),
    nn.Linear(7*7*32, 2*z_dim),
    lambda x: torch.chunk(x, chunks=2, dim=1),
    lambda mean, rho: dists.Normal(mean, 1e-10+torch.log1p(torch.exp(rho)))
])

d = Chain([
    nn.Linear(z_dim, 7*7*32),
    nn.LeakyReLU(),
    lambda x: torch.reshape(x, (-1, 32, 7, 7)),
    nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(),
    nn.ConvTranspose2d(16, 8, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(),
    nn.Conv2d(8, 1, kernel_size=3, padding=1)
])

class VAE(nn.Module):
    def __init__(self, z_dim=64, levels=2):
        super().__init__()

    def sample(self):
        in_channels, out_channels = 1, 1
        combine = Chain([
            lambda z, h: torch.cat((z, h), dim=1),
            nn.Conv2d(2*in_channels, in_channels, 1),
            nn.LeakyReLU()
        ])
        upscale = Chain([
            nn.ConvTranspose2d()
            nn.LeakyReLU()
        ])
        condition_on = Chain([

        ])
        Chain([
            lambda q, h: (q.rsample(), h),
            combine,
            upscale,
            Split(condition_on, nn.Identity), 
        ])

    

In [56]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
transform = transforms.ToTensor()
# data_train = datasets.MNIST(root="", train=True, transform=transform)
# data_test = datasets.MNIST(root="", train=False, transform=transform)
# loader_train = DataLoader(data_train, batch_size=1, shuffle=True)

vae = VAE()
x = torch.zeros(16, 1, 28, 28)
vae(x)